# Data Cleaning Vision System

In [46]:
# %pip install langchain --quiet
# %pip install pypdf --quiet
# %pip install openai --quiet
# %pip install unstructured --quiet

In [47]:
%pip install tiktoken --quiet
%pip install jq --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [48]:
# !pip install "docarray[hnswlib]" --quiet
# !pip install faiss-cpu --quiet


In [49]:
import os

from pathlib import Path
import json

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

from langchain.document_loaders import JSONLoader as Loader

from langchain.indexes import VectorstoreIndexCreator

from langchain.vectorstores import DocArrayHnswSearch

import config

## Data Cleaning 

In [50]:
file = Path('./VisionSystem/visionsystem_scrap.json')
assert file.exists()


In [51]:
data = json.loads(Path(file).read_text())
len(data)

50

In [52]:
# Clean Step #1 - Remove null texts
for item in data:
    if not isinstance(item['text'], str):
        data.pop(data.index(item))

len(data)

50

In [53]:
# # Clean Step #2 - Remove all uninformative text before AGENDAR ONLINE
# for item in data:
    
#     text = item.get('text')
#     descr = item.get('description')
      
#     if text is None:
#         continue 
    
#     item['text'] = text.split('AGENDAR ONLINE')[1]

    

# # Clean Step #3 - Remove
# # disclosure_text = 'Responsável ' + json_file[8]['text'].split('\nResponsável ')[1]
# for item in data:
    
#     text = item.get('text')
#     descr = item.get('description')
      
#     if text is None:
#         continue 
    
#     item['text'] = text.split('\nResponsável')[0]
    
# # json_file.append({'description': 'Informações Gerais', 'text': disclosure_text})

In [58]:
with open(file.with_name('visionsystem_cleaned.json'), 'w') as json_file:
    json.dump(data, json_file)
    

In [56]:
with open(file.with_name('visionsystem_cleaned_small.json'), 'w') as json_file:
    json.dump(data[:20], json_file)
    

## Create the Index

In [57]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

from langchain.embeddings.openai import OpenAIEmbeddings


In [59]:
file_path = file.with_name('visionsystem_cleaned_small.json').as_posix()
file_path = file.with_name('visionsystem_cleaned.json').as_posix()


assert (Path(file_path).exists())

In [60]:
loader = Loader(file_path=file_path, jq_schema='.[].text', text_content=True)

In [61]:
docs = loader.load()
len(docs)

50

In [62]:
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(
    docs, embeddings,)

In [63]:
index = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
).from_loaders([loader])

In [66]:
query = "a Vision tem software de CRM? qual o nome?"

response = index.query(query)

display(Markdown(response))

 Sim, a Vision tem software de CRM. O nome do software é Vision Controller.

In [67]:
index.vectorstore.save_local('./VisionSystem/vision_faiss.store')

## Loading Pre-saved VectorStore

In [208]:
vectorstore = FAISS.load_local('./HammerHead/hh_faiss.store', embeddings=embeddings)

In [209]:
new_index = VectorStoreIndexWrapper(vectorstore=vectorstore)

In [212]:
new_index.query('olá')

' Olá! Não entendi a sua pergunta.'

In [214]:
new_index.query('Vocês possuem óculos?')

' Sim, temos o Óculos Focus Júnior 2.0.'

In [215]:
new_index.query('Qual o preço?')

' R$146,90'

In [216]:
new_index.query('Possuem apenas este modelo?')

' Não, existem vários modelos disponíveis.'

In [217]:
new_index.query('Quais são os modelos disponíveis?')

' Os modelos disponíveis são o Velocity 4.0 e o Vortex 2.0.'

In [54]:
new_index.query('Quantos óculos de natação que voces possuem?')

' Nós possuímos dois óculos de natação: o Óculos de Natação Aqua 2.0 da linha F.SHERER e o Óculos de Natação AQUATECH MIRROR da linha performance.'

In [55]:
new_index.query('Qual a marca vocês representam?')

' Representamos a marca Hammerhead.'

In [56]:
new_index.query('Como é o óculos Aqua 2.0?')

' O óculos Aqua 2.0 da linha F.SHERER é perfeito para aqueles que praticam a natação no dia a dia. Possui lentes de policarbonato, 100% de proteção UV, narigueira removível em 3 tamanhos (ajustáveis), tira dupla de silicone (mais resistente e confortável) e padrão ótico internacional.'

In [57]:
new_index.query('Qual o preço do óculos Aqua 2.0')

' O óculos Aqua 2.0 custa R$90,90.'

In [59]:
new_index.query('Vocês entregam em todo o brasil?')

' Não, não entregamos em todo o Brasil. No entanto, trabalhamos com empresas parceiras, provedores de serviços ou parceiros para gerenciar ou suportar certos aspectos de nossas operações comerciais em nosso nome, com o objetivo de nos ajudar a gerenciar a loja. Esses provedores de serviços ou parceiros podem estar localizados nos Estados Unidos, na Argentina, no Brasil ou em outros locais globais.'

In [48]:
new_index.query('Como é o óculos Olympic?')

' O óculos Olympic possui lentes de policarbonato, 100% de proteção contra os raios UV, narigueira removível em três tamanhos, vedação em silicone Comfort, tira de silicone Racing e design com baixo perfil.'

In [51]:
new_index.query('Qual o preço do óculos Olympic?')

' O preço do óculos Olympic é R$84,90.'

In [62]:
new_index.query('Vocês têm óculos de natação infantil?')

' Sim, temos óculos de natação infantil. O Óculos Focus Júnior 2.0 e o Óculos de Natação Sprinter Júnior são ótimas opções para crianças que praticam natação. O Óculos Focus Júnior 1.0 e o Óculos de Natação Aqua 2.0 também são ótimas opções.'

In [63]:
new_index.query('Que tipo de raquete vocês possuem?')

' Possuímos uma raquete de Beach Tennis Hammerhead Avenger. Ela possui uma moldura de carbono e uma face de carbono 3K, acompanhada de uma capa protetora.'

In [64]:
new_index.query('Quais raquetes vocês possuem?')

' Nós possuímos a raquete de Beach Tennis Hammerhead Avenger. Ela possui uma combinação de carbono e fibra de vidro na estrutura, proporcionando leveza, absorção de impacto e controle nos golpes. Ela também possui tecnologias como Quadro tubular duplo em carbono, Revestimento em fibra de vidro S2-Glass, EVA Soft 15, Spin Surface, 28 furos, Baixo impacto e conforto, Excelente elasticidade e resistência. O comprimento é de 50cm, o peso é de 330g, o perfil é de 22mm e o ponto de equilíbrio é de 25cm. Acompanha capa protetora.'

In [65]:
new_index.query('Vocês possuem a raquete warrior?')

' Sim, possuímos a raquete Hammerhead Warrior. Ela possui quadro tubular duplo em carbono e face em carbono 3K com 3 mil filamentos.'

## Adding Docs To Database

In [322]:
query = "Quais produtos estão disponíveis"
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

Lista de Produtos. Esta página apresenta todas as categorias de produtos que estão disponíveis na loja HammerHead.Os produtos estão divididos nas seguintes categorias:Roupas de natação, Acessórios de natação, raquetes de beach tennis.


In [320]:
product_doc = 'Lista de Produtos. Esta página apresenta todas as categorias de produtos que estão disponíveis na loja HammerHead.'
product_doc += 'Os produtos estão divididos nas seguintes categorias:'
product_doc += 'Roupas de natação, Acessórios de natação, raquetes de beach tennis.' 


In [321]:
vectorstore.add_texts(texts=[product_doc])

['98ace042-5a92-49fb-adff-a7ca3d7e50a4']

In [331]:
vectorstore.save_local('./HammerHead/hh_faiss.store')

## Creating RetrievalQA with Memory

In [323]:
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
import langchain

In [324]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [335]:
qa = ConversationalRetrievalChain.from_llm(
    llm=OpenAI(temperature=0), 
    retriever=vectorstore.as_retriever(top_k=10), 
    memory=memory,
    return_source_documents=False,
    verbose=False

)

In [336]:
langchain.debug=False
result = qa({"question": 'Qual marca você representa?'})
langchain.debug=False
result['answer']

' Represento a marca Hammerhead.'

In [337]:
result = qa({"question": 'Quais categorias de produtos vocês possuem?'})
result['answer']

' As categorias de produtos da marca Hammerhead são Roupas de natação, Acessórios de natação e Raquetes de beach tennis.'

In [338]:
result = qa({"question": 'Quais óculos de natação vocês possuem?'})
result['answer']

' A marca Hammerhead oferece os óculos de natação Hammerhead Eclipse, Hammerhead Avenger, Hammerhead Wave Pro e Shark.'

In [339]:
result = qa({"question": 'Quanto custa o Hammerhead Avenger?'})
result['answer']

' R$96,90'

In [340]:
result = qa({"question": 'Eu gostaria de comprar o Hammerhead Avenger?'})
result['answer']

' R$699,00'

In [ ]:
Quanto custa o sunquíni hammerhead?

In [ ]:
result = qa({"question": 'Vocês possuem óculos de natação para crianças?'})

In [334]:
langchain.debug = True
result = qa({"question": 'Quanto custa o sunquíni hammerhead?'})
result['answer']

[chain/start] [1:chain:ConversationalRetrievalChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "question": "Quanto custa o sunquíni hammerhead?",
  "chat_history": "\nHuman: Qual marca você representa?\nAssistant:  Represento a marca Hammerhead.\nHuman: Qual marca você representa?\nAssistant:  Represento a marca Hammerhead.\nHuman: Quais categorias de produtos vocês possuem?\nAssistant:  A marca Hammerhead possui as categorias de produtos Roupas de natação, Acessórios de natação e raquetes de beach tennis.\nHuman: Quais os preços do óculos Sprinter JR?\nAssistant:  Não sei.\nHuman: Quantos reais custa o óculos Sprinter JR?\nAssistant:  Não sei."
}
[llm/start] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Given the following conversation and a follow up question, rephrase the follow up question to be a stan

' Não sei.'

In [242]:
ConversationalRetrievalChain.from_llm?

Signature:
ConversationalRetrievalChain.from_llm(
    llm: 'BaseLanguageModel',
    retriever: 'BaseRetriever',
    condense_question_prompt: 'BasePromptTemplate' = PromptTemplate(input_variables=['chat_history', 'question'], output_parser=None, partial_variables={}, template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:', template_format='f-string', validate_template=True),
    chain_type: 'str' = 'stuff',
    verbose: 'bool' = False,
    condense_question_llm: 'Optional[BaseLanguageModel]' = None,
    combine_docs_chain_kwargs: 'Optional[Dict]' = None,
    callbacks: 'Callbacks' = None,
    **kwargs: 'Any',
) -> 'BaseConversationalRetrievalChain'
Docstring:
Convenience method to load chain from LLM and retriever.

This provides some logic to create the `question_generator` chain
as well as the combine_

In [239]:
vectorstore.as_retriever(top_k=10)

VectorStoreRetriever(tags=None, metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0xffff4b3f86a0>, search_type='similarity', search_kwargs={})

In [237]:
ConversationalRetrievalChain?

Init signature:
ConversationalRetrievalChain(
    *,
    memory: Optional[langchain.schema.memory.BaseMemory] = None,
    callbacks: Union[List[langchain.callbacks.base.BaseCallbackHandler], langchain.callbacks.base.BaseCallbackManager, NoneType] = None,
    callback_manager: Optional[langchain.callbacks.base.BaseCallbackManager] = None,
    verbose: bool = None,
    tags: Optional[List[str]] = None,
    metadata: Optional[Dict[str, Any]] = None,
    combine_docs_chain: langchain.chains.combine_documents.base.BaseCombineDocumentsChain,
    question_generator: langchain.chains.llm.LLMChain,
    output_key: str = 'answer',
    rephrase_question: bool = True,
    return_source_documents: bool = False,
    return_generated_question: bool = False,
    get_chat_history: Optional[Callable[[Union[Tuple[str, str], langchain.schema.messages.BaseMessage]], str]] = None,
    retriever: langchain.schema.retriever.BaseRetriever,
    max_tokens_limit: Optional[int] = None,
) -> None
Docstring:     
C

In [219]:
from langchain.memory import ConversationBufferMemory, ChatMessageHistory

In [220]:
from langchain.schema.messages import SystemMessage

In [221]:
# first we have the database 
type(db), type(vectorstore)

(langchain.vectorstores.faiss.FAISS, langchain.vectorstores.faiss.FAISS)

In [222]:
docs = db.similarity_search('Quais óculos de natação voces possuem?', k=10)
len(docs)

10

In [223]:
retriever = db.as_retriever()

In [224]:
llm = ChatOpenAI(temperature = 0.0)


In [225]:
memory = ChatMessageHistory()

In [226]:
conversationmemory = ConversationBufferMemory()

In [227]:
# conversationmemory.chat_memory.add_user_message('Olá')

In [228]:
# conversationmemory.chat_memory.add_ai_message('Olá, eu sou a inteligência artificial da HammerHead. Em que posso ajudá-lo')

In [229]:
conversationmemory.clear()
s = 'Você vai atuar como um assistente virtual da marca HammerHead. Inicie a conversa '
s += 'informand o nome da marca (HammerHead) e explicando que você está disponível para tirar dúvidas e dar sugestões sobre produtos da marca. '
s += 'Responda sempre de forma cordial e educada. '

conversationmemory.chat_memory.add_message(SystemMessage(content=s)) 
                                                         

In [230]:
conversationmemory.buffer

'System: Você vai atuar como um assistente virtual da marca HammerHead. Inicie a conversa informand o nome da marca (HammerHead) e explicando que você está disponível para tirar dúvidas e dar sugestões sobre produtos da marca. Responda sempre de forma cordial e educada. '

In [231]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    verbose=True,
    memory=conversationmemory,
    top_k=10
)

ValidationError: 1 validation error for RetrievalQA
top_k
  extra fields not permitted (type=value_error.extra)

In [163]:
qa.return_source_documents = True

In [206]:
response = qa('Quais os preços?')

In [201]:
response.keys()

dict_keys(['query', 'result'])

In [202]:
response['history']

KeyError: 'history'

In [207]:
response['result']

'Os preços dos produtos não estão especificados no contexto fornecido.'

In [183]:
docs = response['source_documents']

KeyError: 'source_documents'

In [161]:
len(docs)

10

In [87]:
RetrievalQA.from_chain_type?

Signature:
RetrievalQA.from_chain_type(
    llm: 'BaseLanguageModel',
    chain_type: 'str' = 'stuff',
    chain_type_kwargs: 'Optional[dict]' = None,
    **kwargs: 'Any',
) -> 'BaseRetrievalQA'
Docstring: Load chain from chain type.
File:      /opt/conda/lib/python3.10/site-packages/langchain/chains/retrieval_qa/base.py
Type:      method

[Document(page_content='Descrição\nO óculos de Natação Aqua 2.0 da linha F.SHERER é perfeito para aqueles que praticam a natação no dia a dia. As narigueiras que acompanham o óculos têm 3 tamanhos ajustáveis, para que o nadador possa ajustar adequadamente. Além disso, possuem tira dupla de silicone e padrão ótico internacional.\nCARACTERÍSTICA\nLentes de policarbonato.\n100% de proteção UV.\nNarigueira removível em 3 tamanhos (ajustáveis).\nTira dupla de silicone (mais resistente e confortável).\nPadrão ótico internacional.\nCOMPOSIÇÃO\nLentes: Policarbonato.\nVedação: PVC.\nTira: Silicone.\nNarigueira: Poliuretano.\nCUIDADOS\n- Não esfregue o interior das lentes.\n- Enxágue com água fria para remover a sujeira e deixe secar ao ar livre.\n- Não deixe seus óculos sob a luz solar direta, isso pode danificar a pigmentação dos óculos.', metadata={'source': '/home/jovyan/LangChain/HammerHead/hammerhead_full_scrap.json', 'seq_num': 113}),
 Document(page_content='Descrição\nO óculos de Nataçã

In [78]:
qa.return_source_documents = True

In [98]:
embeddings = OpenAIEmbeddings()
db = DocArrayInMemorySearch.from_documents(
    docs, embeddings, n_dim=1536
)

In [100]:
db.similarity_search(query)

[Document(page_content=' \nFacebook-f Instagram Google \nAGENDAR CONSULTA \nMenu \nMenu \nPesquisar \nExcelência técnica \ncom olhar humano.\nAGENDE UMA CONSULTA\nPrecisão e segurança\npara o seu diagnóstico.\nVER EXAMES\nCirurgias Oculares\nVER CIRURGIAS\nSua opinião é \nimportante para nós.\nDEIXE SUA AVALIAÇÃO\nExcelência técnica \ncom olhar humano.\nAGENDE UMA CONSULTA\nPrecisão e segurança\npara o seu diagnóstico.\nVER EXAMES\nCirurgias Oculares\nVER CIRURGIAS\nSua opinião é \nimportante para nós.\nDEIXE SUA AVALIAÇÃO\nExcelência técnica \ncom olhar humano.\nAGENDE UMA CONSULTA\nPrecisão e segurança\npara o seu diagnóstico.\nVER EXAMES\nCirurgias Oculares\nVER CIRURGIAS\nSua opinião é \nimportante para nós.\nDEIXE SUA AVALIAÇÃO\nAnterior \nPróximo \nSobre o Instituto\nCom mais de 30 anos de tradição e localizado no centro de Florianópolis, no Instituto de Olhos de Florianópolis você receberá um atendimento especial e personalizado para cuidar da saúde dos seus olhos.. \nSAIBA MAIS

In [15]:
query = "Que tipos de informação estão disponíveis no FAQ?"
response = index.query(query)
display(Markdown(response))

 O FAQ fornece informações sobre como limpar óculos de natação, diferenças entre as lentes, tecnologias, armações, vedações e narigueiras, e como usar um spray ou líquido antifog.

In [16]:
query = "Como limpar o óculos?"
response = index.query(query)
display(Markdown(response))

 O primeiro passo para higienizar/limpar os óculos é enxaguá-los com água filtrada e gelada após nadar em qualquer tipo de água. Tanto o sal quanto o cloro podem causar danos nas lentes, tiras, presilhas e na película antiembaçante, portanto, trate de enxaguar seus óculos com água limpa após cada uso. Agora é a hora de secar os óculos. O correto é deixar os óculos secarem naturalmente e fora da luz solar, pois o calor pode derreter as partes de silicone e trincar a lente. Certifique-se de que seus óculos estão totalmente secos antes de guardá-los, caso contrário bactérias e fungos podem danificar o produto. Por último, mas não menos importante, guarde seus ó

In [18]:
docs = loader.load()

In [21]:
docs[1]

IndexError: list index out of range

In [24]:
docs = loader.load_and_split()